# TF-IDF

En este notebook se transforma el texto preprocesado (`text_clean`) en una representación numérica usando TF-IDF.

In [ ]:
import pandas as pd #armar y guardar el CSV
import pickle # Se usa para guardar los objetos entrenados
from sklearn.feature_extraction.text import TfidfVectorizer

Se definen los respositorios a utilizar

In [2]:
repos_slugs = ["auto1111_webui", "ytdlp"]

**Iteración por repositorio**

* Se recorre cada slug en repos_slugs.
* Cada repositorio se procesa de forma independiente.

**Carga del texto limpio**

* Se lee el archivo messages_clean.csv. (Contiene la columna text_clean ya preprocesada)

**Configuración del vectorizador TF-IDF**

* Estos parámetros reducen ruido y mejoran la representación.

**Vectorización del texto**

* Se aplica fit_transform sobre text_clean.(Se obtiene la matriz X_tfidf (documentos × términos))

**Persistencia por repositorio**

* Se guarda el TfidfVectorizer entrenado (tfidf_vectorizer.pkl).
* Se guarda la matriz TF-IDF (X_tfidf.pkl).
* Permite reutilizar los vectores sin recalcular.

**Guardado del dataset listo**

* Se guarda messages_clean_ready.csv. (Dataset alineado con la matriz TF-IDF)

X_tfidf es una matriz dispersa de (n_documentos, n_terminos)

In [3]:
for slug in repos_slugs: 
    df = pd.read_csv(f"../data/{slug}/messages_clean.csv") #ruta donde está el archivo generado en el file 2

    vectorizer = TfidfVectorizer( #Crear el vectorizador TF-IDF
        ngram_range=(1, 2), # usa unigramas y bigramas.
        min_df=2, # ignora términos que aparecen solo una vez (Errores, Nombres muy específicos, Ruido)
        max_df=0.95 #elimina términos demasiado frecuentes (No ayuda a diferenciar documentos)
    ) #Estos parámetros reducen ruido y mejoran la representación.

    X_tfidf = vectorizer.fit_transform(df["text_clean"])
    #Aprende el vocabulario y construye el diccionario de términos
    #Con transform convierte cada doc en un vevtor numérico

    # Guardar todo por repo
    with open(f"../data/{slug}/tfidf_vectorizer.pkl", "wb") as f: #Se guarda el modelo TF-IDF entrenado
        pickle.dump(vectorizer, f)

    with open(f"../data/{slug}/X_tfidf.pkl", "wb") as f: #Se guarda la matriz numérica
        pickle.dump(X_tfidf, f) #Esto permite usarla después para: Similitud, Clustering y Modelos

    df.to_csv(f"../data/{slug}/messages_clean_ready.csv", index=False)

    print(f"\nRepo {slug}: X_tfidf shape = {X_tfidf.shape}") #Se guarda una versión final limpia
    print("Guardado vectorizer + X_tfidf + messages_clean_ready.csv")


Repo auto1111_webui: X_tfidf shape = (459, 8618)
Guardado vectorizer + X_tfidf + messages_clean_ready.csv

Repo ytdlp: X_tfidf shape = (697, 9603)
Guardado vectorizer + X_tfidf + messages_clean_ready.csv
